#Libraries and Data Find Future list

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt
from xgboost import plot_tree

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
findFeature = False
#tickers = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]

features = ["Avg_Range"]
features.append("TARGET")
symbol = "^NSEI"
df = pd.read_csv("../stock_historical_data/{}.csv".format(symbol))
df.set_index("Date", inplace=True)
features

['Avg_Range', 'TARGET']

In [3]:
#strat_mgr = StrategyManager(symbol, "", "",concatenated_df)
data = DataProcessing(symbol,df)
print(data)

/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*i

In [4]:
df_f = data.df.copy()


In [5]:
df_f

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets,RSI,RSI_Ret,MA_12,MA_21,Roll_Rets,Avg_Range,Returns_T1,Range_T1,RSI_Ret_T1,Returns_T2,Range_T2,RSI_Ret_T2
Date,,,,,,,,,,,,,,,,,,,,
2013-04-29,-2.687478,-0.857076,-0.867172,5904.100098,-1.310941,0.005561,0.386621,-1.0,56.873757,0.983333,5730.737508,5753.211937,0.161021,-0.156026,0.013603,0.288672,1.090742,0.000428,-1.311491,1.002975
2013-05-02,-0.048176,3.881063,4.192497,5999.350098,-1.378812,0.011661,0.303455,-1.0,62.174531,1.093202,5757.137533,5766.528599,0.163494,-0.188867,0.005561,0.386621,0.983333,0.013603,0.288672,1.090742
2013-05-06,1.269143,-1.413804,-1.038976,5971.049805,-5.065963,0.004551,12.838596,-1.0,59.821824,0.962160,5779.358358,5779.507161,0.172144,0.313399,0.011661,0.303455,1.093202,0.005561,0.386621,0.983333
2013-05-07,-1.799699,-4.121651,-33.073542,6043.549805,-1.973705,0.012142,0.346882,-1.0,63.619800,1.063488,5803.979167,5791.854771,0.189355,0.292789,0.004551,12.838596,0.962160,0.011661,0.303455,1.093202
2013-05-08,1.079875,-0.558843,-0.236378,6069.299805,-1.434796,0.004261,-0.221885,-1.0,64.889280,1.019954,5847.841634,5801.664295,0.189436,0.302974,0.012142,0.346882,1.063488,0.004551,12.838596,0.962160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,-1.613425,-1.953067,-1.002986,18691.199219,-7.692646,0.001377,84.290311,-1.0,61.740370,0.875659,18662.024902,18492.666667,0.105103,2.674456,0.002134,-0.732807,1.014003,0.003266,-0.292459,0.992899
2023-06-27,-2.116168,-4.126554,-169.715818,18817.400391,-2.601690,0.006752,0.580556,-1.0,65.282167,1.057366,18703.379069,18519.409505,0.106771,2.686028,0.001377,84.290311,0.875659,0.002134,-0.732807,1.014003
2023-06-28,1.402315,0.688108,1.169755,18972.099609,1.651000,0.008221,0.229700,-1.0,69.062869,1.057913,18742.774902,18562.414249,0.114116,2.683200,0.006752,0.580556,1.057366,0.001377,84.290311,0.875659


In [6]:
# Specify Target
df_f.loc[df_f["Range"].shift(-1) > df_f["Avg_Range"], "TARGET"] = 1
df_f.loc[df_f["Range"].shift(-1) <= df_f["Avg_Range"], "TARGET"] = 0

In [7]:
# Check for NaNs
nan_location = np.where(np.isnan(df_f))
nan_location
# Fill NA
df_f["TARGET"].fillna(0, inplace=True)
df_f.tail()

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets,RSI,RSI_Ret,...,MA_21,Roll_Rets,Avg_Range,Returns_T1,Range_T1,RSI_Ret_T1,Returns_T2,Range_T2,RSI_Ret_T2,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-26,-1.613425,-1.953067,-1.002986,18691.199219,-7.692646,0.001377,84.290311,-1.0,61.740370,0.875659,...,18492.666667,0.105103,2.674456,0.002134,-0.732807,1.014003,0.003266,-0.292459,0.992899,0.0
2023-06-27,-2.116168,-4.126554,-169.715818,18817.400391,-2.601690,0.006752,0.580556,-1.0,65.282167,1.057366,...,18519.409505,0.106771,2.686028,0.001377,84.290311,0.875659,0.002134,-0.732807,1.014003,0.0
2023-06-28,1.402315,0.688108,1.169755,18972.099609,1.651000,0.008221,0.229700,-1.0,69.062869,1.057913,...,18562.414249,0.114116,2.683200,0.006752,0.580556,1.057366,0.001377,84.290311,0.875659,0.0
2023-06-30,0.048093,0.036407,0.101137,19189.050781,-1.477542,0.011435,0.157412,-1.0,73.432364,1.063268,...,18606.445219,0.125228,2.586896,0.008221,0.229700,1.057913,0.006752,0.580556,1.057366,0.0
2023-07-03,-0.003255,-0.254509,0.274122,19322.550781,-0.767595,0.006957,-0.322797,-1.0,75.706160,1.030964,...,18656.759580,0.132208,2.586301,0.011435,0.157412,1.063268,0.008221,0.229700,1.057913,0.0


In [8]:
# Remove unwanted columns
df_tts = df_f.copy()
df_tts.drop(columns=["Close", "Bench_C_Rets", "Open", "High", "Low"], inplace=True)
# Find columns with infinite values
columns_with_inf = df_tts.columns[np.isinf(df_tts).any()]

#Drop columns with infinite values
df_tts.drop(columns_with_inf, axis=1, inplace=True)
has_inf = np.isinf(df_tts.values).any()

if has_inf:
    print("DataFrame contains NaN values",columns_with_inf)
else:
    print("DataFrame does not contain NaN values")

DataFrame does not contain NaN values


In [9]:

# Feature Selection
df_tts = df_tts.copy()
if findFeature == False:
    df_tts = df_tts[features]
df_tts

,Avg_Range,TARGET
Date,,
2013-04-29,-0.156026,1.0
2013-05-02,-0.188867,1.0
2013-05-06,0.313399,1.0
2013-05-07,0.292789,0.0
2013-05-08,0.302974,1.0
...,...,...
2023-06-26,2.674456,0.0
2023-06-27,2.686028,0.0
2023-06-28,2.683200,0.0


In [10]:
# Perform Train Test Split
# Split into Learning (X) and Target (y) Data
X = df_tts.iloc[:, : -1]
y = df_tts.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)

# Select type of model to optimize for
is_binary = True
is_optimise_for_precision = True
# Determine Objective and Eval Metrics
if is_binary:
    objective = "binary:logistic"
    eval_metric = "logloss"
    eval_metric_list = ["error", "logloss", eval_metric]
else:
    objective = "multi:softmax"
    eval_metric = "mlogloss"
    eval_metric_list = ["merror", "mlogloss", eval_metric]
# Refine Eval Metric
if is_binary and is_optimise_for_precision:
    eval_metric = "aucpr"
    scoring = "precision"
elif is_binary and not is_optimise_for_precision:
    eval_metric = "auc"
    scoring = "f1"
else:
    scoring = "accuracy"
# Build First Classifier Model 0
classifier_0 = XGBClassifier(
    objective=objective,
    booster="gbtree",
    eval_metric=eval_metric,
    subsample=0.8,
    colsample_bytree=1,
    random_state=1,
    use_label_encoder=False
)
# Provide Gris for Hyperparams
param_grid = {
    "gamma": [0, 0.1, 0.2, 0.5, 1, 1.5, 2, 3, 6, 12, 20],
    "learning_rate": [0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8],
    "max_depth": [1, 2, 3, 4, 5, 6, 8, 12],
    "n_estimators": [25, 50, 65, 80, 100, 115, 200]
}
grid_search = RandomizedSearchCV(estimator=classifier_0, param_distributions=param_grid, scoring=scoring)

# Perform Random Search for Best Hyper params
best_model = grid_search.fit(X_train, y_train)
hyperparams = best_model.best_params_
ne = hyperparams["n_estimators"]
lr = hyperparams["learning_rate"]
md = hyperparams["max_depth"]
gm = hyperparams["gamma"]
print("Recommended Params >>", f"ne: {ne},", f"lr: {lr}", f"md: {md}", f"gm: {gm}")

Shape of X_train:  (1081, 1)
Shape of y_train:  (1081,)


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  w

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  w

Recommended Params >> ne: 25, lr: 0.3 md: 3 gm: 20


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [11]:
# Build Classification Model 1
classifier_1 = XGBClassifier(
    objective=objective,
    booster="gbtree",
    eval_metric=eval_metric,
    n_estimators=ne,
    learning_rate=lr,
    max_depth=md,
    gamma=gm,
    subsample=0.8,
    colsample_bytree=1,
    random_state=1,
    use_label_encoder=False
)

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [12]:
# Fit Model
eval_set = [(X_train, y_train)]
classifier_1.set_params(eval_metric=eval_metric_list)  # Example metric: 'error'

classifier_1.fit(
    X_train,
    y_train,
    #eval_metric=eval_metric_list,
    eval_set=eval_set,
    verbose=False
)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=20, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [13]:
# Get predictions for training data
train_yhat = classifier_1.predict(X_train)
print("Training Preds: \n", train_yhat[:5])

Training Preds: 
 [1 0 0 0 0]


In [14]:
X_train

,Avg_Range
Date,
2020-09-15,-0.556008
2020-07-03,0.845862
2014-03-20,0.230059
2018-09-14,2.732040
2022-05-16,2.144926
...,...
2021-08-02,-0.169858
2021-10-14,-2.766191
2023-01-23,-0.664681


In [15]:
# Set K-Fold Cross Validation Levels
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [16]:
train_results = cross_val_score(classifier_1, X_train, y_train, scoring=scoring, cv=cv, n_jobs=1)

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  w

In [17]:
# Brief Review of Training Results
print("Average Accuracy K-Fold: ", round(train_results.mean(), 2))
print("Std Deviation K-Fold: ", round(train_results.std(), 2))
print("Precision Score 0: ", round(precision_score(y_train, train_yhat, average=None)[0], 3))
print("Precision Score 1: ", round(precision_score(y_train, train_yhat, average=None)[1], 3))
print("")
print("Just for reference. Right now, we are only focussed on getting some initial features.")
print("If the results look too good to be true, they probably are.")

Average Accuracy K-Fold:  0.7
Std Deviation K-Fold:  0.03
Precision Score 0:  0.778
Precision Score 1:  0.707

Just for reference. Right now, we are only focussed on getting some initial features.
If the results look too good to be true, they probably are.


In [18]:
classifier_1.fit(
    X,
    y,
    #eval_metric=eval_metric_list,
    eval_set=eval_set,
    verbose=False
)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=20, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [1]:
# Plot Feature Importances
fig = plt.figure(figsize=(22, 5))
importance_labels = X.columns
importance_features = classifier_1.feature_importances_
plt.bar(importance_labels, importance_features)
plt.show()

NameError: name 'plt' is not defined

In [20]:
# Select Best Features
mean_feature_importance = importance_features.mean()
i = 0
recommended_feature_labels = []
recommended_feature_score = []
for fi in importance_features:
    if fi > mean_feature_importance:
        recommended_feature_labels.append(importance_labels[i])
        recommended_feature_score.append(fi)
    i += 1

In [21]:
recommended_feature_labels

[]

In [22]:
# Save the trained model to a file
if findFeature == False:
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(classifier_1, f)

In [23]:
# Save the trained model to a file
if findFeature == True:
    print(recommended_feature_labels)
    with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'wb') as f:
        pickle.dump(recommended_feature_labels, f)

In [ ]:
rcParams["figure.figsize"] = 80, 50
plot_tree(classifier)